In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import time, os
import pickle
import re
from tqdm import tqdm # tool to track progress

In [2]:
df = pd.read_csv("../Data/LinkedIn_Job_Postings_notcleaned.csv")
df.head()

,Date,Company,Title,Job Link,criteria
0,2022-01-15,Intel Corporation,Platform Supply Analyst,https://www.linkedin.com/jobs/view/platform-su...,Seniority level\nMid-Senior level\nEmployment ...
1,2022-01-26,Intel Corporation,Platform Supply Analyst,https://www.linkedin.com/jobs/view/platform-su...,\n \n \n Seniority ...
2,2022-01-10,Kaiser Permanente,Supervisor Support Services,https://www.linkedin.com/jobs/view/supervisor-...,\n \n \n Seniority ...
3,2022-02-02,Dignity Health,Communications Specialist,https://www.linkedin.com/jobs/view/communicati...,\n \n \n Seniority ...
4,2022-01-26,Intel Corporation,Planning Analyst,https://www.linkedin.com/jobs/view/planning-an...,\n \n \n Seniority ...


In [ ]:
# the criteria column is really hard to clean
# trying to find ways to scrape the same info without cleaning after

In [70]:
#checking error website
r = requests.get("https://www.linkedin.com/jobs/view/americas-customer-enablement-and-customer-program-analyst-at-intel-corporation-2884909824/?refId=Urp6CPmklNV9kVKZYpiq3g%3D%3D&trackingId=LxNPZZyN%2F2j%2BGAN2PRF2kA%3D%3D&position=10&pageNum=0&trk=public_jobs_jserp-result_search-card")
s = bs(r.text, 'html5lib')
criteria = s.find("ul", class_="description__job-criteria-list")
criteria.find_all("li")
criteria_dict = {} # setup a dictionary for each url
for i in criteria_list:
    item = i.findChild()  # gets feature name
    info = item.findNext().text.strip() # get feature info
    criteria_dict[item.text.strip()] = info  

In [75]:
detail_list = []
for url in tqdm(df['Job Link']):
    r = requests.get(url)
    s = bs(r.text, 'html5lib')
    criteria = s.find("ul", class_="description__job-criteria-list")
    try:
        criteria_list = criteria.find_all("li")
        criteria_dict = {}
        for i in criteria_list:
            item = i.findChild()
            info = item.findNext().text.strip()
            criteria_dict[item.text.strip()] = info
        detail_list.append(criteria_dict)
    except:
        print(url)
    time.sleep(5)


100%|██████████| 998/998 [1:32:14<00:00,  5.55s/it]


In [76]:
len(detail_list)

998

In [77]:
criteria_df = pd.DataFrame(detail_list)
criteria_df.head()

,Seniority level,Employment type,Job function,Industries
0,Mid-Senior level,Full-time,Management and Manufacturing,"IT Services and IT Consulting, Appliances, Ele..."
1,Mid-Senior level,Full-time,Management and Manufacturing,"IT Services and IT Consulting, Appliances, Ele..."
2,Mid-Senior level,Full-time,Information Technology,"Financial Services, Wellness and Fitness Servi..."
3,Entry level,Full-time,"Marketing, Public Relations, and Writing/Editing",Hospitals and Health Care
4,Mid-Senior level,Full-time,Business Development and Sales,"IT Services and IT Consulting, Appliances, Ele..."


In [78]:
criteria_df['Job Link'] = df['Job Link']
criteria_df.head()

,Seniority level,Employment type,Job function,Industries,Job Link
0,Mid-Senior level,Full-time,Management and Manufacturing,"IT Services and IT Consulting, Appliances, Ele...",https://www.linkedin.com/jobs/view/platform-su...
1,Mid-Senior level,Full-time,Management and Manufacturing,"IT Services and IT Consulting, Appliances, Ele...",https://www.linkedin.com/jobs/view/platform-su...
2,Mid-Senior level,Full-time,Information Technology,"Financial Services, Wellness and Fitness Servi...",https://www.linkedin.com/jobs/view/supervisor-...
3,Entry level,Full-time,"Marketing, Public Relations, and Writing/Editing",Hospitals and Health Care,https://www.linkedin.com/jobs/view/communicati...
4,Mid-Senior level,Full-time,Business Development and Sales,"IT Services and IT Consulting, Appliances, Ele...",https://www.linkedin.com/jobs/view/planning-an...


In [79]:
final_df = pd.merge(df, criteria_df, on= ['Job Link'])

In [80]:
final_df.head()

,Date,Company,Title,Job Link,criteria,Seniority level,Employment type,Job function,Industries
0,2022-01-15,Intel Corporation,Platform Supply Analyst,https://www.linkedin.com/jobs/view/platform-su...,Seniority level\nMid-Senior level\nEmployment ...,Mid-Senior level,Full-time,Management and Manufacturing,"IT Services and IT Consulting, Appliances, Ele..."
1,2022-01-26,Intel Corporation,Platform Supply Analyst,https://www.linkedin.com/jobs/view/platform-su...,\n \n \n Seniority ...,Mid-Senior level,Full-time,Management and Manufacturing,"IT Services and IT Consulting, Appliances, Ele..."
2,2022-01-10,Kaiser Permanente,Supervisor Support Services,https://www.linkedin.com/jobs/view/supervisor-...,\n \n \n Seniority ...,Mid-Senior level,Full-time,Information Technology,"Financial Services, Wellness and Fitness Servi..."
3,2022-02-02,Dignity Health,Communications Specialist,https://www.linkedin.com/jobs/view/communicati...,\n \n \n Seniority ...,Entry level,Full-time,"Marketing, Public Relations, and Writing/Editing",Hospitals and Health Care
4,2022-01-26,Intel Corporation,Planning Analyst,https://www.linkedin.com/jobs/view/planning-an...,\n \n \n Seniority ...,Mid-Senior level,Full-time,Business Development and Sales,"IT Services and IT Consulting, Appliances, Ele..."


In [82]:
final_df.drop(columns=['criteria'],inplace=True)
final_df.head()

,Date,Company,Title,Job Link,Seniority level,Employment type,Job function,Industries
0,2022-01-15,Intel Corporation,Platform Supply Analyst,https://www.linkedin.com/jobs/view/platform-su...,Mid-Senior level,Full-time,Management and Manufacturing,"IT Services and IT Consulting, Appliances, Ele..."
1,2022-01-26,Intel Corporation,Platform Supply Analyst,https://www.linkedin.com/jobs/view/platform-su...,Mid-Senior level,Full-time,Management and Manufacturing,"IT Services and IT Consulting, Appliances, Ele..."
2,2022-01-10,Kaiser Permanente,Supervisor Support Services,https://www.linkedin.com/jobs/view/supervisor-...,Mid-Senior level,Full-time,Information Technology,"Financial Services, Wellness and Fitness Servi..."
3,2022-02-02,Dignity Health,Communications Specialist,https://www.linkedin.com/jobs/view/communicati...,Entry level,Full-time,"Marketing, Public Relations, and Writing/Editing",Hospitals and Health Care
4,2022-01-26,Intel Corporation,Planning Analyst,https://www.linkedin.com/jobs/view/planning-an...,Mid-Senior level,Full-time,Business Development and Sales,"IT Services and IT Consulting, Appliances, Ele..."


In [83]:
final_df.to_csv("../Data/linkedin_job_posts_cleanish.csv")

In [84]:
final_df['Seniority level'].value_counts()

Entry level         439
Associate           343
Mid-Senior level     95
Internship           33
Not Applicable       32
Director             23
Executive            11
Name: Seniority level, dtype: int64

In [85]:
final_df['Employment type'].value_counts()

Full-time     758
Contract      122
Temporary      86
Part-time      20
Internship     10
Other           1
Volunteer       1
Name: Employment type, dtype: int64

In [86]:
final_df['Job function'].value_counts()

Management and Manufacturing                                129
Finance and Sales                                            88
Other                                                        67
Engineering and Information Technology                       65
Information Technology                                       63
Human Resources                                              59
Research, Analyst, and Information Technology                51
Marketing and Sales                                          50
Marketing, Public Relations, and Writing/Editing             47
Sales and Business Development                               45
Business Development and Sales                               45
Purchasing and Supply Chain                                  43
Administrative                                               43
Customer Service and Information Technology                  32
Project Management and Information Technology                29
Legal                                   

In [87]:
final_df['Industries'].value_counts()

IT Services and IT Consulting                                                    49
Hospitals and Health Care                                                        37
Design Services                                                                  32
Non-profit Organizations                                                         27
Legal Services                                                                   27
                                                                                 ..
Higher Education, Non-profit Organizations, and Hospitals and Health Care         1
Medical Equipment Manufacturing and Biotechnology Research                        1
Construction, Transportation, Logistics and Storage, and Truck Transportation     1
Software Development, Human Resources Services, and Financial Services            1
Higher Education                                                                  1
Name: Industries, Length: 180, dtype: int64